In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr

2025-07-18 12:11:28.862328: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 12:11:28.898471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-18 12:11:28.898498: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-18 12:11:28.899384: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-18 12:11:28.906501: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# We also need the trained model
base_model_non_overlapping = tf.keras.models.load_model("models/stage2_base_model_overlapping.keras")

2025-07-18 12:11:30.656619: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [3]:
base_model_non_overlapping.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 13, None, None, 3)   0         []                            
                             ]                                                                    
                                                                                                  
 time_distributed (TimeDist  (None, 13, 32)               71808     ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 lstm (LSTM)                 [(None, 64),                 24832     ['time_distributed[0][0]']    
                              (None, 64),                                                   

In [4]:
# Where is the data?

In [5]:
# features
Test_input_features_loc = "data/Overlapping_data/Test_input_features"
Test_input_features_loc_content = os.listdir(Test_input_features_loc)
Test_input_features_loc_content.sort()

In [6]:
Test_input_features_loc_content

['test_data_blk_0103.npy',
 'test_data_blk_0104.npy',
 'test_data_blk_0105.npy',
 'test_data_blk_0106.npy',
 'test_data_blk_0201.npy',
 'test_data_blk_0202.npy',
 'test_data_blk_0205.npy',
 'test_data_blk_0206.npy',
 'test_data_blk_0302.npy',
 'test_data_blk_0303.npy',
 'test_data_blk_0304.npy',
 'test_data_blk_0305.npy',
 'test_data_blk_0306.npy']

In [7]:
block_0103_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[0]))

In [8]:
block_0103_features.shape

(12288, 13, 32, 32, 3)

In [9]:
# test targets
test_targets_loc = "data/Stage2_base_model/Overlapping/test_targets"

In [10]:
test_target_loc_contents = os.listdir(test_targets_loc)
test_target_loc_contents.sort()

In [11]:
test_target_loc_contents

['block_0103_targets.npy',
 'block_0104_targets.npy',
 'block_0105_targets.npy',
 'block_0106_targets.npy',
 'block_0201_targets.npy',
 'block_0202_targets.npy',
 'block_0205_targets.npy',
 'block_0206_targets.npy',
 'block_0302_targets.npy',
 'block_0303_targets.npy',
 'block_0304_targets.npy',
 'block_0305_targets.npy',
 'block_0306_targets.npy']

In [12]:
block_0103_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[0]))

In [13]:
block_0103_targets.shape

(12288, 7)

In [14]:
# np.sum(block_0103_targets, axis = 0)

In [15]:
# We cannot get the true counts as above in this case as we are using overlapping sub windows. Maybe we can get them from the previously stored datasets?

In [16]:
# Okay, e need to get the predictions for the test sub-images properly. We have previously defined some functions for this, let's reuse them here. I think these can be used in the overlapping case too. We will also need to find a way to include the true test counts (per image), because the true test targets do not even 

In [17]:
%%time
# To get the predictions on a block, we can just do the model.predict function
block_0103_preds = base_model_non_overlapping.predict(block_0103_features)

2025-07-18 12:11:37.126290: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


384/384 [==============================] - 5s 11ms/step
CPU times: user 8.47 s, sys: 1.71 s, total: 10.2 s
Wall time: 8.67 s


In [18]:
block_0103_preds.shape

(12288, 7, 1)

In [19]:
# reshape these to shape of the targets
block_0103_preds_reshaped = block_0103_preds.reshape(block_0103_preds.shape[0], block_0103_preds.shape[1])

In [20]:
block_0103_preds_reshaped.shape

(12288, 7)

In [21]:
# now we need to normalize the final preds - account for overlapping. 

In [22]:
# preds_block_0103_final_alt = np.sum(block_0103_preds_reshaped, axis = 0)

In [23]:
# # We do need some good values - even to say that the model is working
# preds_block_0103_final_alt

In [24]:
# Let's do the function for post-hoc normalization as below - but this seems to be for a single image

In [25]:
def prediction_on_test_data(pred_values, image_height, image_width, stride = 8, kernel_size = 32):
    # density map
    Density_map = np.zeros((image_height, image_width))

    # counts map
    counts_map = np.zeros((image_height, image_width))
    
    # now, for every window, we will keep adding the values together and also add the counts
    counter = 0
#     need a counter to move into each predicted value in the pred values list
    for ii in range(0, image_height, stride):
        for jj in range(0, image_width, stride):
#         operations for density map
#             get the window of interest
            new_window = Density_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     fill each with the value c_k
            counts_window = np.full((new_window.shape[0], new_window.shape[1]), pred_values[counter])
#     get the shapes of this new window
            cw_height = counts_window.shape[0]
            cw_width = counts_window.shape[1]
#         Do c_k/r_2
            counts_window_new = counts_window/(cw_height*cw_width)
#     This is the value in the window now
            value_window = counts_window_new
#     place the values in the corrsponding area of the density map
            Density_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window + value_window

#         Let's now focus on capturing the counts of the windows
            new_window_c = counts_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     get the counts area
            count = np.ones((new_window_c.shape[0], new_window_c.shape[1]))
#     keep adding the counts to reflect the addition of densities
            counts_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window_c + count
#     increase the counter
            counter = counter + 1
            
#         get the normalized count
    normalized_counts = np.divide(Density_map, counts_map)
    
#     entire count on the test set
    pred_on_test = np.sum(normalized_counts)
    
#     return the predicted value
    return(pred_on_test, normalized_counts)


In [26]:
image_height = 768
image_width = 1024

print(image_height) ,print(image_width)

768
1024


(None, None)

In [27]:
block_0103_preds_reshaped[:, 0].shape

(12288,)

In [30]:
# Let's get the preds using the function - for may be the first image
preds_block_0103_im_0 = prediction_on_test_data(block_0103_preds_reshaped[:, 0], image_height, image_width, 8, 32)

In [35]:
# preds_block_0103_im_0

In [32]:
preds_block_0103_im_0[0]

55.52982107421848

In [33]:
preds_block_0103_im_0[1]

array([[4.81684510e-05, 4.81684510e-05, 4.81684510e-05, ...,
        8.45954728e-05, 8.45954728e-05, 8.45954728e-05],
       [4.81684510e-05, 4.81684510e-05, 4.81684510e-05, ...,
        8.45954728e-05, 8.45954728e-05, 8.45954728e-05],
       [4.81684510e-05, 4.81684510e-05, 4.81684510e-05, ...,
        8.45954728e-05, 8.45954728e-05, 8.45954728e-05],
       ...,
       [1.14373302e-04, 1.14373302e-04, 1.14373302e-04, ...,
        1.92701479e-04, 1.92701479e-04, 1.92701479e-04],
       [1.14373302e-04, 1.14373302e-04, 1.14373302e-04, ...,
        1.92701479e-04, 1.92701479e-04, 1.92701479e-04],
       [1.14373302e-04, 1.14373302e-04, 1.14373302e-04, ...,
        1.92701479e-04, 1.92701479e-04, 1.92701479e-04]])

In [34]:
np.sum(preds_block_0103_im_0[1])

55.52982107421848

In [32]:
# So we need a second function it seems to get the pred values for all images in the test block. May be we can compute the metrics using that function too. Let's use the next function we have already written

In [36]:
block_0103_preds_reshaped.shape

(12288, 7)

In [37]:
block_0103_targets.shape

(12288, 7)

In [63]:
def get_final_forecasted_and_true_values(preds_from_model, true_targets, im_height, im_weight, stride, kernel_size, true_csv_file_loc_location, new_csv_loc, blk_name):
    final_preds_list = []
    for i in range(preds_from_model.shape[1]):
        preds_per_image = prediction_on_test_data(preds_from_model[:,i], im_height, im_weight, stride , kernel_size)
        final_preds_list.append(preds_per_image[0])
    # make this list  a dataframe
    final_preds =  np.array(final_preds_list)
    
    # True test counts
    # read the csv file
    read_csv_file_here = pd.read_csv(os.path.join(true_csv_file_loc_location, blk_name + '.csv'))
    true_densities = np.array(read_csv_file_here['True_densities'])
    
    # compute the mae
    mae_value = mean_absolute_error(true_densities, final_preds)
    # compute the rmse
    rmse_value = np.sqrt(mean_squared_error(true_densities, final_preds))
    # pearsonr
    pearson_value = pearsonr(true_densities, final_preds)
    # r2score
    r2score_value = r2_score(true_densities, final_preds)
    
    # attach the true and the forecasted values together
    # give also a name for the test image
    im_names = np.array(["test_im_" + str(i) for i in range(7)])
    final_df = pd.DataFrame(zip(im_names, true_densities, final_preds), columns = ["Image_name","True_densities", "Predicted_densities"])
    # save the csv file for later use
    final_df.to_csv(os.path.join(new_csv_loc, blk_name + '.csv'), index = False)
    all_metrics = [mae_value, rmse_value, pearson_value, r2score_value]

    return all_metrics, final_df 

In [45]:
csv_file_loc = "data/Stage2_base_model/Non_overlapping/predicted_and_true_densities/"
new_loc = "data/Stage2_base_model/Overlapping/predicted_and_true_densities/"

In [46]:
file_name = 'true_and_pred_densities_blk_0103.csv'

In [47]:
read_csv_file = pd.read_csv(os.path.join(csv_file_loc, file_name))

In [48]:
np.array(read_csv_file['True_densities']).shape

(7,)

In [52]:
%%time
all_preds_0103 = []
for i in range(block_0103_preds_reshaped.shape[1]):
    
    preds = prediction_on_test_data(block_0103_preds_reshaped[:, i], image_height, image_width, 8, 32)[0]
    all_preds_0103.append(preds)

CPU times: user 1.11 s, sys: 730 µs, total: 1.11 s
Wall time: 1.11 s


In [53]:
all_preds_0103

[55.52982107421848,
 56.6467927643108,
 51.964747331251125,
 45.50522409149077,
 39.15900831825801,
 33.65191214733771,
 29.15938688375252]

In [54]:
np.array(all_preds_0103)

array([55.52982107, 56.64679276, 51.96474733, 45.50522409, 39.15900832,
       33.65191215, 29.15938688])

In [55]:
np.array(all_preds_0103).shape

(7,)

In [51]:
# Okay, now let's see if our functions work

In [58]:
im_height = 768
# lol why did we right weight, but let's go with this else it is too much to fix
im_weight = 1024
stride = 8
kernel_size = 32
block_name = 'true_and_pred_densities_blk_0103'

In [59]:
csv_file_loc = "data/Stage2_base_model/Non_overlapping/predicted_and_true_densities/"
new_loc = "data/Stage2_base_model/Overlapping/predicted_and_true_densities/"

In [65]:
file_name = 'true_and_pred_densities_blk_0103'

In [66]:
blk_0103_metrics, blk_0103_df = get_final_forecasted_and_true_values(block_0103_preds_reshaped, block_0103_targets, im_height, im_weight, stride, kernel_size, csv_file_loc, new_loc, file_name)

In [67]:
blk_0103_metrics

[10.616331005566161,
 11.828715612291235,
 PearsonRResult(statistic=0.6285898482575835, pvalue=0.13053265317194954),
 -4.225043893397271]

In [68]:
blk_0103_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,40.000661,55.529821
1,test_im_1,39.000001,56.646793
2,test_im_2,41.000000,51.964747
3,test_im_3,31.000000,45.505224
4,test_im_4,32.000000,39.159008
5,test_im_5,40.002086,33.651912
6,test_im_6,27.000176,29.159387


In [56]:
# Okay, so things seem to work okay, let's decide what all we want to do for the rest of the test blocks

In [57]:
# We should be able to do this in a loop, but for simplicity,and keepign track of things, let's do this blockwise

In [58]:
# Block 0104

In [59]:
# load the input features and targets

In [60]:
# features
Test_input_features_loc_content[1]

'test_data_blk_0104.npy'

In [61]:
block_0104_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[1]))

In [62]:
block_0104_features.shape

(910, 13, 30, 30, 3)

In [63]:
# targets
test_target_loc_contents[1]

'block_0104_targets.npy'

In [64]:
block_0104_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[1]))

In [65]:
block_0104_targets.shape

(910, 7)

In [66]:
np.sum(block_0104_targets, axis = 0)

array([33.        , 30.        , 39.00000127, 40.        , 40.99881044,
       42.1690088 , 30.00531667])

In [67]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0104_preds = base_model_non_overlapping.predict(block_0104_features)

29/29 [==============================] - 0s 6ms/step


In [68]:
block_0104_preds.shape

(910, 7, 1)

In [69]:
# reshape the preds
block_0104_preds_reshaped = block_0104_preds.reshape(block_0104_preds.shape[0], block_0104_preds.shape[1])

In [70]:
block_0104_preds_reshaped.shape

(910, 7)

In [71]:
block_0104_final_preds_alt = np.sum(block_0104_preds_reshaped, axis = 0)

In [72]:
block_0104_final_preds_alt

array([21.040163 , 23.76801  , 21.9592   , 18.571503 , 15.492011 ,
       13.4325285, 12.196832 ], dtype=float32)

In [73]:
block_name_0104 = 'true_and_pred_densities_blk_0104'

In [74]:
block_0104_metrics, blk_0104_df = get_final_forecasted_and_true_values(block_0104_preds_reshaped, block_0104_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0104)

In [75]:
block_0104_metrics

[18.387554912057606,
 19.730211923046785,
 PearsonRResult(statistic=-0.25249410725821214, pvalue=0.5848848473770533),
 -15.264333082125898]

In [76]:
blk_0104_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,33.000000,21.040162
1,test_im_1,30.000000,23.768009
2,test_im_2,39.000001,21.959211
3,test_im_3,40.000000,18.571495
4,test_im_4,40.998810,15.492018
5,test_im_5,42.169009,13.432526
6,test_im_6,30.005317,12.196831


In [77]:
# verify mae
mae_df_0104 = mean_absolute_error(blk_0104_df['True_densities'], blk_0104_df['Predicted_densities'])

In [78]:
mae_df_0104

18.387554912057606

In [79]:
# Okay, I think we can do this for the rest of the blocks

In [80]:
# Block 0105

In [81]:
# load the input features and targets

In [82]:
# features
Test_input_features_loc_content[2]

'test_data_blk_0105.npy'

In [83]:
block_0105_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[2]))

In [84]:
block_0105_features.shape

(910, 13, 30, 30, 3)

In [85]:
# targets
test_target_loc_contents[2]

'block_0105_targets.npy'

In [86]:
block_0105_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[2]))

In [87]:
block_0105_targets.shape

(910, 7)

In [88]:
np.sum(block_0105_targets, axis = 0)

array([40.00000127, 46.00274308, 58.00069611, 41.000032  , 41.00118956,
       36.00002186, 23.00000049])

In [89]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0105_preds = base_model_non_overlapping.predict(block_0105_features)

29/29 [==============================] - 0s 6ms/step


In [90]:
block_0105_preds.shape

(910, 7, 1)

In [91]:
# reshape the preds
block_0105_preds_reshaped = block_0105_preds.reshape(block_0105_preds.shape[0], block_0105_preds.shape[1])

In [92]:
block_0105_preds_reshaped.shape

(910, 7)

In [93]:
block_0105_final_preds_alt = np.sum(block_0105_preds_reshaped, axis = 0)

In [94]:
block_0105_final_preds_alt

array([26.957481, 32.198895, 29.99872 , 25.06695 , 20.220856, 16.579393,
       14.183497], dtype=float32)

In [95]:
block_name_0105 = 'true_and_pred_densities_blk_0105'

In [96]:
block_0105_metrics, blk_0105_df = get_final_forecasted_and_true_values(block_0105_preds_reshaped, block_0105_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0105)

In [97]:
block_0105_metrics

[17.114121622900903,
 18.068677485546477,
 PearsonRResult(statistic=0.8224933359725033, pvalue=0.0231148562842959),
 -2.44446322367713]

In [98]:
blk_0105_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,40.000001,26.957487
1,test_im_1,46.002743,32.198901
2,test_im_2,58.000696,29.998726
3,test_im_3,41.000032,25.066958
4,test_im_4,41.001190,20.220861
5,test_im_5,36.000022,16.579399
6,test_im_6,23.000000,14.183502


In [99]:
# verify mae
mae_df_0105 = mean_absolute_error(blk_0105_df['True_densities'], blk_0105_df['Predicted_densities'])

In [100]:
mae_df_0105

17.114121622900903

In [101]:
# Block 0106

In [102]:
# load the input features and targets

In [103]:
# features
Test_input_features_loc_content[3]

'test_data_blk_0106.npy'

In [104]:
block_0106_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[3]))

In [105]:
block_0106_features.shape

(910, 13, 30, 30, 3)

In [106]:
# targets
test_target_loc_contents[3]

'block_0106_targets.npy'

In [107]:
block_0106_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[3]))

In [108]:
block_0106_targets.shape

(910, 7)

In [109]:
np.sum(block_0106_targets, axis = 0)

array([38.99966666, 38.9999891 , 45.        , 39.98688727, 42.99999681,
       47.99650163, 38.        ])

In [110]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0106_preds = base_model_non_overlapping.predict(block_0106_features)

29/29 [==============================] - 0s 6ms/step


In [111]:
block_0106_preds.shape

(910, 7, 1)

In [112]:
# reshape the preds
block_0106_preds_reshaped = block_0106_preds.reshape(block_0106_preds.shape[0], block_0106_preds.shape[1])

In [113]:
block_0106_preds_reshaped.shape

(910, 7)

In [114]:
block_0106_final_preds_alt = np.sum(block_0106_preds_reshaped, axis = 0)

In [115]:
block_0106_final_preds_alt

array([25.447723, 31.891478, 30.85111 , 26.700651, 22.235804, 18.754534,
       16.4021  ], dtype=float32)

In [116]:
block_name_0106 = 'true_and_pred_densities_blk_0106'

In [117]:
block_0106_metrics, blk_0106_df = get_final_forecasted_and_true_values(block_0106_preds_reshaped, block_0106_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0106)

In [118]:
block_0106_metrics

[17.099946426585,
 18.375505925304324,
 PearsonRResult(statistic=-0.11100962683232617, pvalue=0.81270323810515),
 -27.32998336251571]

In [119]:
blk_0106_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,38.999667,25.447734
1,test_im_1,38.999989,31.891481
2,test_im_2,45.000000,30.851123
3,test_im_3,39.986887,26.700641
4,test_im_4,42.999997,22.235805
5,test_im_5,47.996502,18.754533
6,test_im_6,38.000000,16.402100


In [120]:
# verify mae
mae_df_0106 = mean_absolute_error(blk_0106_df['True_densities'], blk_0106_df['Predicted_densities'])

In [121]:
mae_df_0106

17.099946426585

In [122]:
# Block 0201

In [123]:
# load the input features and targets

In [124]:
# features
Test_input_features_loc_content[4]

'test_data_blk_0201.npy'

In [125]:
block_0201_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[4]))

In [126]:
block_0201_features.shape

(910, 13, 30, 30, 3)

In [127]:
# targets
test_target_loc_contents[4]

'block_0201_targets.npy'

In [128]:
block_0201_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[4]))

In [129]:
block_0201_targets.shape

(910, 7)

In [130]:
np.sum(block_0201_targets, axis = 0)

array([45.00021729, 45.0000397 , 47.00000127, 37.99999951, 42.00004097,
       35.        , 29.        ])

In [131]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0201_preds = base_model_non_overlapping.predict(block_0201_features)

29/29 [==============================] - 0s 6ms/step


In [132]:
block_0201_preds.shape

(910, 7, 1)

In [133]:
# reshape the preds
block_0201_preds_reshaped = block_0201_preds.reshape(block_0201_preds.shape[0], block_0201_preds.shape[1])

In [134]:
block_0201_preds_reshaped.shape

(910, 7)

In [135]:
block_0201_final_preds_alt = np.sum(block_0201_preds_reshaped, axis = 0)

In [136]:
block_0201_final_preds_alt

array([31.223482, 39.00834 , 36.34357 , 29.907192, 23.876047, 19.396769,
       16.392017], dtype=float32)

In [137]:
block_name_0201 = 'true_and_pred_densities_blk_0201'

In [138]:
block_0201_metrics, blk_0201_df = get_final_forecasted_and_true_values(block_0201_preds_reshaped, block_0201_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0201)

In [139]:
block_0201_metrics

[12.121834944538023,
 12.737191218493807,
 PearsonRResult(statistic=0.8733729653050728, pvalue=0.010224795613735899),
 -3.4912328138338102]

In [140]:
blk_0201_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,45.000217,31.223487
1,test_im_1,45.000040,39.008347
2,test_im_2,47.000001,36.343591
3,test_im_3,38.000000,29.907183
4,test_im_4,42.000041,23.876053
5,test_im_5,35.000000,19.396763
6,test_im_6,29.000000,16.392031


In [141]:
# verify mae
mae_df_0201 = mean_absolute_error(blk_0201_df['True_densities'], blk_0201_df['Predicted_densities'])

In [142]:
mae_df_0201

12.121834944538023

In [143]:
# Block 0202

In [144]:
# load the input features and targets

In [145]:
# features
Test_input_features_loc_content[5]

'test_data_blk_0202.npy'

In [146]:
block_0202_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[5]))

In [147]:
block_0202_features.shape

(910, 13, 30, 30, 3)

In [148]:
# targets
test_target_loc_contents[5]

'block_0202_targets.npy'

In [149]:
block_0202_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[5]))

In [150]:
block_0202_targets.shape

(910, 7)

In [151]:
np.sum(block_0202_targets, axis = 0)

array([17.9999603 , 21.        , 22.99999951, 20.99998216, 21.        ,
       18.        , 18.        ])

In [152]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0202_preds = base_model_non_overlapping.predict(block_0202_features)

29/29 [==============================] - 0s 6ms/step


In [153]:
block_0202_preds.shape

(910, 7, 1)

In [154]:
# reshape the preds
block_0202_preds_reshaped = block_0202_preds.reshape(block_0202_preds.shape[0], block_0202_preds.shape[1])

In [155]:
block_0202_preds_reshaped.shape

(910, 7)

In [156]:
block_0202_final_preds_alt = np.sum(block_0202_preds_reshaped, axis = 0)

In [157]:
block_0202_final_preds_alt

array([19.253525, 29.330297, 28.616823, 22.801538, 16.61203 , 12.142984,
        9.477439], dtype=float32)

In [158]:
block_name_0202 = 'true_and_pred_densities_blk_0202'

In [159]:
block_0202_metrics, blk_0202_df = get_final_forecasted_and_true_values(block_0202_preds_reshaped, block_0202_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0202)

In [160]:
block_0202_metrics

[5.109972381951524,
 5.756395031073505,
 PearsonRResult(statistic=0.791208344408407, pvalue=0.03406801679471176),
 -8.66464269158945]

In [161]:
blk_0202_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,17.999960,19.253531
1,test_im_1,21.000000,29.330298
2,test_im_2,23.000000,28.616809
3,test_im_3,20.999982,22.801555
4,test_im_4,21.000000,16.612028
5,test_im_5,18.000000,12.142981
6,test_im_6,18.000000,9.477437


In [162]:
# verify mae
mae_df_0202 = mean_absolute_error(blk_0202_df['True_densities'], blk_0202_df['Predicted_densities'])

In [163]:
mae_df_0202

5.109972381951524

In [164]:
# Block 0205

In [165]:
# load the input features and targets

In [166]:
# features
Test_input_features_loc_content[6]

'test_data_blk_0205.npy'

In [167]:
block_0205_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[6]))

In [168]:
block_0205_features.shape

(910, 13, 30, 30, 3)

In [169]:
# targets
test_target_loc_contents[6]

'block_0205_targets.npy'

In [170]:
block_0205_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[6]))

In [171]:
block_0205_targets.shape

(910, 7)

In [172]:
np.sum(block_0205_targets, axis = 0)

array([44.        , 42.00000127, 45.        , 43.        , 39.        ,
       40.99991511, 31.99965576])

In [173]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0205_preds = base_model_non_overlapping.predict(block_0205_features)

29/29 [==============================] - 0s 6ms/step


In [174]:
block_0205_preds.shape

(910, 7, 1)

In [175]:
# reshape the preds
block_0205_preds_reshaped = block_0205_preds.reshape(block_0205_preds.shape[0], block_0205_preds.shape[1])

In [176]:
block_0205_preds_reshaped.shape

(910, 7)

In [177]:
block_0205_final_preds_alt = np.sum(block_0205_preds_reshaped, axis = 0)

In [178]:
block_0205_final_preds_alt

array([26.88361 , 32.78388 , 30.425714, 25.263613, 20.396631, 17.081272,
       15.063345], dtype=float32)

In [179]:
block_name_0205 = 'true_and_pred_densities_blk_0205'

In [180]:
block_0205_metrics, blk_0205_df = get_final_forecasted_and_true_values(block_0205_preds_reshaped, block_0205_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0205)

In [181]:
block_0205_metrics

[16.871643417587713,
 17.36137838073463,
 PearsonRResult(statistic=0.7581816991785636, pvalue=0.04824648683497765),
 -17.368997688752348]

In [182]:
blk_0205_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,44.000000,26.883602
1,test_im_1,42.000001,32.783880
2,test_im_2,45.000000,30.425719
3,test_im_3,43.000000,25.263618
4,test_im_4,39.000000,20.396628
5,test_im_5,40.999915,17.081278
6,test_im_6,31.999656,15.063344


In [183]:
# verify mae
mae_df_0205 = mean_absolute_error(blk_0205_df['True_densities'], blk_0205_df['Predicted_densities'])

In [184]:
mae_df_0205

16.871643417587713

In [185]:
# Block 0206

In [186]:
# load the input features and targets

In [187]:
# features
Test_input_features_loc_content[7]

'test_data_blk_0206.npy'

In [188]:
block_0206_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[7]))

In [189]:
block_0206_features.shape

(910, 13, 30, 30, 3)

In [190]:
# targets
test_target_loc_contents[7]

'block_0206_targets.npy'

In [191]:
block_0206_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[7]))

In [192]:
block_0206_targets.shape

(910, 7)

In [193]:
np.sum(block_0206_targets, axis = 0)

array([40.99983792, 41.99881044, 39.00006705, 32.00000319, 25.00035194,
       23.0000397 , 17.99999951])

In [194]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0206_preds = base_model_non_overlapping.predict(block_0206_features)

29/29 [==============================] - 0s 6ms/step


In [195]:
block_0206_preds.shape

(910, 7, 1)

In [196]:
# reshape the preds
block_0206_preds_reshaped = block_0206_preds.reshape(block_0206_preds.shape[0], block_0206_preds.shape[1])

In [197]:
block_0206_preds_reshaped.shape

(910, 7)

In [198]:
block_0206_final_preds_alt = np.sum(block_0206_preds_reshaped, axis = 0)

In [199]:
block_0206_final_preds_alt

array([28.086142 , 34.561092 , 31.860779 , 25.888538 , 20.087923 ,
       15.830046 , 13.1231785], dtype=float32)

In [200]:
block_name_0206 = 'true_and_pred_densities_blk_0206'

In [201]:
block_0206_metrics, blk_0206_df = get_final_forecasted_and_true_values(block_0206_preds_reshaped, block_0206_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0206)

In [202]:
block_0206_metrics

[7.2230581658326924,
 7.650646136742023,
 PearsonRResult(statistic=0.966901638205503, pvalue=0.0003760327680105684),
 0.25999617278398446]

In [203]:
blk_0206_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,40.999838,28.086145
1,test_im_1,41.998810,34.561102
2,test_im_2,39.000067,31.860777
3,test_im_3,32.000003,25.888535
4,test_im_4,25.000352,20.087923
5,test_im_5,23.000040,15.830037
6,test_im_6,18.000000,13.123183


In [204]:
# verify mae
mae_df_0206 = mean_absolute_error(blk_0206_df['True_densities'], blk_0206_df['Predicted_densities'])

In [205]:
mae_df_0206

7.2230581658326924

In [206]:
# Block 0302

In [207]:
# load the input features and targets

In [208]:
# features
Test_input_features_loc_content[8]

'test_data_blk_0302.npy'

In [209]:
block_0302_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[8]))

In [210]:
block_0302_features.shape

(910, 13, 30, 30, 3)

In [211]:
# targets
test_target_loc_contents[8]

'block_0302_targets.npy'

In [212]:
block_0302_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[8]))

In [213]:
block_0302_targets.shape

(910, 7)

In [214]:
np.sum(block_0302_targets, axis = 0)

array([49.        , 49.00000499, 53.9995696 , 50.04234944, 43.00000897,
       48.00057246, 36.99999903])

In [215]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0302_preds = base_model_non_overlapping.predict(block_0302_features)

29/29 [==============================] - 0s 6ms/step


In [216]:
block_0302_preds.shape

(910, 7, 1)

In [217]:
# reshape the preds
block_0302_preds_reshaped = block_0302_preds.reshape(block_0302_preds.shape[0], block_0302_preds.shape[1])

In [218]:
block_0302_preds_reshaped.shape

(910, 7)

In [219]:
block_0302_final_preds_alt = np.sum(block_0302_preds_reshaped, axis = 0)

In [220]:
block_0302_final_preds_alt

array([32.35997 , 40.169575, 37.364464, 30.80883 , 24.407537, 19.720293,
       16.628996], dtype=float32)

In [221]:
block_name_0302 = 'true_and_pred_densities_blk_0302'

In [222]:
block_0302_metrics, blk_0302_df = get_final_forecasted_and_true_values(block_0302_preds_reshaped, block_0302_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0302)

In [223]:
block_0302_metrics

[18.36897975677104,
 19.130958004327876,
 PearsonRResult(statistic=0.7715286113935932, pvalue=0.04218881137756312),
 -12.992434686637706]

In [224]:
blk_0302_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,49.000000,32.359977
1,test_im_1,49.000005,40.169574
2,test_im_2,53.999570,37.364463
3,test_im_3,50.042349,30.808824
4,test_im_4,43.000009,24.407526
5,test_im_5,48.000572,19.720290
6,test_im_6,36.999999,16.628991


In [225]:
# verify mae
mae_df_0302 = mean_absolute_error(blk_0302_df['True_densities'], blk_0302_df['Predicted_densities'])

In [226]:
mae_df_0302

18.36897975677104

In [227]:
# Block 0303

In [228]:
# load the input features and targets

In [229]:
# features
Test_input_features_loc_content[9]

'test_data_blk_0303.npy'

In [230]:
block_0303_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[9]))

In [231]:
block_0303_features.shape

(910, 13, 30, 30, 3)

In [232]:
# targets
test_target_loc_contents[9]

'block_0303_targets.npy'

In [233]:
block_0303_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[9]))

In [234]:
block_0303_targets.shape

(910, 7)

In [235]:
np.sum(block_0303_targets, axis = 0)

array([49.0001712 , 46.00000673, 42.99998216, 38.99999312, 36.02588424,
       36.        , 23.        ])

In [236]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0303_preds = base_model_non_overlapping.predict(block_0303_features)

29/29 [==============================] - 0s 6ms/step


In [237]:
block_0303_preds.shape

(910, 7, 1)

In [238]:
# reshape the preds
block_0303_preds_reshaped = block_0303_preds.reshape(block_0303_preds.shape[0], block_0303_preds.shape[1])

In [239]:
block_0303_preds_reshaped.shape

(910, 7)

In [240]:
block_0303_final_preds_alt = np.sum(block_0303_preds_reshaped, axis = 0)

In [241]:
block_0303_final_preds_alt

array([29.802235, 36.192806, 33.718746, 28.110909, 22.660995, 18.519194,
       15.768269], dtype=float32)

In [242]:
block_name_0303 = 'true_and_pred_densities_blk_0303'

In [243]:
block_0303_metrics, blk_0303_df = get_final_forecasted_and_true_values(block_0303_preds_reshaped, block_0303_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0303)

In [244]:
block_0303_metrics

[12.464693654951825,
 13.125500280454363,
 PearsonRResult(statistic=0.855520858230087, pvalue=0.014076575657107257),
 -1.7488373576663343]

In [245]:
blk_0303_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,49.000171,29.802246
1,test_im_1,46.000007,36.192800
2,test_im_2,42.999982,33.718760
3,test_im_3,38.999993,28.110912
4,test_im_4,36.025884,22.661001
5,test_im_5,36.000000,18.519200
6,test_im_6,23.000000,15.768262


In [246]:
# verify mae
mae_df_0303 = mean_absolute_error(blk_0303_df['True_densities'], blk_0303_df['Predicted_densities'])

In [247]:
mae_df_0303

12.464693654951825

In [248]:
# Block 0304

In [249]:
# load the input features and targets

In [250]:
# features
Test_input_features_loc_content[10]

'test_data_blk_0304.npy'

In [251]:
block_0304_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[10]))

In [252]:
block_0304_features.shape

(910, 13, 30, 30, 3)

In [253]:
# targets
test_target_loc_contents[10]

'block_0304_targets.npy'

In [254]:
block_0304_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[10]))

In [255]:
block_0304_targets.shape

(910, 7)

In [256]:
np.sum(block_0304_targets, axis = 0)

array([37.        , 41.00205748, 42.99999808, 41.99995481, 37.99999951,
       34.        , 24.        ])

In [257]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0304_preds = base_model_non_overlapping.predict(block_0304_features)

29/29 [==============================] - 0s 6ms/step


In [258]:
block_0304_preds.shape

(910, 7, 1)

In [259]:
# reshape the preds
block_0304_preds_reshaped = block_0304_preds.reshape(block_0304_preds.shape[0], block_0304_preds.shape[1])

In [260]:
block_0304_preds_reshaped.shape

(910, 7)

In [261]:
block_0304_final_preds_alt = np.sum(block_0304_preds_reshaped, axis = 0)

In [262]:
block_0304_final_preds_alt

array([28.013153, 32.7043  , 29.934435, 24.829102, 20.06204 , 16.581528,
       14.351597], dtype=float32)

In [263]:
block_name_0304 = 'true_and_pred_densities_blk_0304'

In [264]:
block_0304_metrics, blk_0304_df = get_final_forecasted_and_true_values(block_0304_preds_reshaped, block_0304_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0304)

In [265]:
block_0304_metrics

[13.217981547218068,
 13.801806325112025,
 PearsonRResult(statistic=0.7998065774071847, pvalue=0.03082041332605045),
 -4.2083817798711705]

In [266]:
blk_0304_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,37.000000,28.013144
1,test_im_1,41.002057,32.704302
2,test_im_2,42.999998,29.934437
3,test_im_3,41.999955,24.829096
4,test_im_4,38.000000,20.062042
5,test_im_5,34.000000,16.581521
6,test_im_6,24.000000,14.351596


In [267]:
# verify mae
mae_df_0304 = mean_absolute_error(blk_0304_df['True_densities'], blk_0304_df['Predicted_densities'])

In [268]:
mae_df_0304

13.217981547218068

In [269]:
# Block 0305

In [270]:
# load the input features and targets

In [271]:
# features
Test_input_features_loc_content[11]

'test_data_blk_0305.npy'

In [272]:
block_0305_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[11]))

In [273]:
block_0305_features.shape

(910, 13, 30, 30, 3)

In [274]:
# targets
test_target_loc_contents[11]

'block_0305_targets.npy'

In [275]:
block_0305_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[11]))

In [276]:
block_0305_targets.shape

(910, 7)

In [277]:
np.sum(block_0305_targets, axis = 0)

array([46.        , 35.        , 37.        , 30.00065157, 35.00118956,
       28.99999357, 20.00001784])

In [278]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0305_preds = base_model_non_overlapping.predict(block_0305_features)

29/29 [==============================] - 0s 6ms/step


In [279]:
block_0305_preds.shape

(910, 7, 1)

In [280]:
# reshape the preds
block_0305_preds_reshaped = block_0305_preds.reshape(block_0305_preds.shape[0], block_0305_preds.shape[1])

In [281]:
block_0305_preds_reshaped.shape

(910, 7)

In [282]:
block_0305_final_preds_alt = np.sum(block_0305_preds_reshaped, axis = 0)

In [283]:
block_0305_final_preds_alt

array([33.38993 , 40.037327, 36.195454, 29.017773, 22.399853, 17.664164,
       14.6303  ], dtype=float32)

In [284]:
block_name_0305 = 'true_and_pred_densities_blk_0305'

In [285]:
block_0305_metrics, blk_0305_df = get_final_forecasted_and_true_values(block_0305_preds_reshaped, block_0305_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0305)

In [286]:
block_0305_metrics

[6.963100188808168,
 8.46952976106841,
 PearsonRResult(statistic=0.7063378079429559, pvalue=0.07604911741839418),
 -0.29797437802799487]

In [287]:
blk_0305_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,46.000000,33.389915
1,test_im_1,35.000000,40.037323
2,test_im_2,37.000000,36.195464
3,test_im_3,30.000652,29.017770
4,test_im_4,35.001190,22.399853
5,test_im_5,28.999994,17.664178
6,test_im_6,20.000018,14.630296


In [288]:
# verify mae
mae_df_0305 = mean_absolute_error(blk_0305_df['True_densities'], blk_0305_df['Predicted_densities'])

In [289]:
mae_df_0305

6.963100188808168

In [290]:
# Block 0306

In [291]:
# load the input features and targets

In [292]:
# features
Test_input_features_loc_content[12]

'test_data_blk_0306.npy'

In [293]:
block_0306_features = np.load(os.path.join(Test_input_features_loc, Test_input_features_loc_content[12]))

In [294]:
block_0306_features.shape

(910, 13, 30, 30, 3)

In [295]:
# targets
test_target_loc_contents[12]

'block_0306_targets.npy'

In [296]:
block_0306_targets = np.load(os.path.join(test_targets_loc, test_target_loc_contents[12]))

In [297]:
block_0306_targets.shape

(910, 7)

In [298]:
np.sum(block_0306_targets, axis = 0)

array([41.00000897, 41.00000319, 43.00685125, 39.99760431, 40.        ,
       32.99998216, 18.        ])

In [299]:
# get preds
# To get the predictions on a block, we can just do the model.predict function
block_0306_preds = base_model_non_overlapping.predict(block_0306_features)

29/29 [==============================] - 0s 6ms/step


In [300]:
block_0306_preds.shape

(910, 7, 1)

In [301]:
# reshape the preds
block_0306_preds_reshaped = block_0306_preds.reshape(block_0306_preds.shape[0], block_0306_preds.shape[1])

In [302]:
block_0306_preds_reshaped.shape

(910, 7)

In [303]:
block_0306_final_preds_alt = np.sum(block_0306_preds_reshaped, axis = 0)

In [304]:
block_0306_final_preds_alt

array([28.099371, 34.962708, 33.061466, 27.824364, 22.603819, 18.6645  ,
       16.0247  ], dtype=float32)

In [305]:
block_name_0306 = 'true_and_pred_densities_blk_0306'

In [306]:
block_0306_metrics, blk_0306_df = get_final_forecasted_and_true_values(block_0306_preds_reshaped, block_0306_targets, im_height, im_weight, stride, kernel_size, csv_file_loc_all, block_name_0306)

In [307]:
block_0306_metrics

[10.680503941701536,
 11.72304806648356,
 PearsonRResult(statistic=0.8037298486445251, pvalue=0.029398715216765844),
 -1.0832344661049222]

In [308]:
blk_0306_df

,Image_name,True_densities,Predicted_densities
0,test_im_0,41.000009,28.099372
1,test_im_1,41.000003,34.962703
2,test_im_2,43.006851,33.061461
3,test_im_3,39.997604,27.824361
4,test_im_4,40.000000,22.603819
5,test_im_5,32.999982,18.664501
6,test_im_6,18.000000,16.024705


In [309]:
# verify mae
mae_df_0306 = mean_absolute_error(blk_0306_df['True_densities'], blk_0306_df['Predicted_densities'])

In [310]:
mae_df_0306

10.680503941701536

In [311]:
# Okay, now let's also compute the metrics for all data taken together, maybe we should stack them all together, and then get the metrics from there

In [312]:
# I think importing them from the stored location would be less hectic than getting all the dfs stored here in a list

In [313]:
dfs_loc = "data/Stage2_base_model/Non_overlapping/predicted_and_true_densities/"

In [314]:
all_dfs = os.listdir(dfs_loc)
all_dfs.sort()

In [315]:
all_dfs

['.ipynb_checkpoints',
 'true_and_pred_densities_blk_0103.csv',
 'true_and_pred_densities_blk_0104.csv',
 'true_and_pred_densities_blk_0105.csv',
 'true_and_pred_densities_blk_0106.csv',
 'true_and_pred_densities_blk_0201.csv',
 'true_and_pred_densities_blk_0202.csv',
 'true_and_pred_densities_blk_0205.csv',
 'true_and_pred_densities_blk_0206.csv',
 'true_and_pred_densities_blk_0302.csv',
 'true_and_pred_densities_blk_0303.csv',
 'true_and_pred_densities_blk_0304.csv',
 'true_and_pred_densities_blk_0305.csv',
 'true_and_pred_densities_blk_0306.csv']

In [316]:
# remove checkpointinf from the list
all_dfs = [file for file in all_dfs if file.split(".")[-1] == 'csv']

In [317]:
all_dfs.sort()
all_dfs

['true_and_pred_densities_blk_0103.csv',
 'true_and_pred_densities_blk_0104.csv',
 'true_and_pred_densities_blk_0105.csv',
 'true_and_pred_densities_blk_0106.csv',
 'true_and_pred_densities_blk_0201.csv',
 'true_and_pred_densities_blk_0202.csv',
 'true_and_pred_densities_blk_0205.csv',
 'true_and_pred_densities_blk_0206.csv',
 'true_and_pred_densities_blk_0302.csv',
 'true_and_pred_densities_blk_0303.csv',
 'true_and_pred_densities_blk_0304.csv',
 'true_and_pred_densities_blk_0305.csv',
 'true_and_pred_densities_blk_0306.csv']

In [318]:
# Okay, now load and stack these?

all_dfs_loaded = [pd.read_csv(os.path.join(dfs_loc, file)) for file in all_dfs]

In [319]:
all_dfs_final = pd.concat(all_dfs_loaded)

In [320]:
all_dfs_final.shape

(91, 3)

In [321]:
13*7

91

In [322]:
all_dfs_final.head()

,Image_name,True_densities,Predicted_densities
0,test_im_0,40.000661,32.156782
1,test_im_1,39.000001,40.467874
2,test_im_2,41.000000,37.502109
3,test_im_3,31.000000,30.661990
4,test_im_4,32.000000,24.149929


In [323]:
# Now get the final metrics
all_test_mae = mean_absolute_error(all_dfs_final['True_densities'], all_dfs_final['Predicted_densities'])
all_test_mae

12.546249705468286

In [324]:
all_test_rmse = np.sqrt(mean_squared_error(all_dfs_final['True_densities'], all_dfs_final['Predicted_densities']))
all_test_rmse

14.259795408289449

In [325]:
all_test_r2score = r2_score(all_dfs_final['True_densities'], all_dfs_final['Predicted_densities'])
all_test_r2score

-1.5338203045376884

In [326]:
all_test_pearsonr = pearsonr(all_dfs_final['True_densities'], all_dfs_final['Predicted_densities'])
all_test_pearsonr[0]

0.5867457226908929